## Building a binary recommendation system with RBM

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

### 1. Importing our data

- It's the famous movie lens dataset you can read more about it [here](https://grouplens.org/datasets/movielens/).
- Here I'm gonna use the 100k old version of the dataset.

In [2]:
# Importing the training dataset and convert it to numpy array
training_set_df = pd.read_csv('dataset/u1.base', delimiter = '\t')
training_set = np.array(training_set_df, dtype=int)

In [3]:
training_set[10]

array([        1,        16,         5, 878543541])

- as shown above the first coulnm refers to the userID, the second one to movieId , the third to the movie rating and the last one refers to time that the user gives the movie the rate 

In [4]:
# Importing the test dataset and convert it to numpy array
test_set_df = pd.read_csv('dataset/u1.test', delimiter='\t')
test_set = np.array(test_set_df, dtype=int)
test_set[10]

array([        1,        36,         2, 875073180])

---
---

### 2. Preparing the data

- The data will be a giant matrix each row of the matrix represent a user and its rating
    > e.g: the first row represent the first user, and the first coulmn in the first row represent the rating from the first user to the first movie
    
    > e.g: matrix[5, 28] -> that will give us the rating from the user number \#5 to the movie \#28
    

- Getting the max user_id, and the max movie_id
    > two know what will be the dimensions of our matrix

#### First we want to get total number of users and the same for movies movies

- In the cell below, We are taking the maximum number from both the test set and the trainning set because the they are randomly suffeled for both the user and  the movie

In [5]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [6]:
# printing the results
print(f'The number of users {nb_users}, The number of movies {nb_movies}')

The number of users 943, The number of movies 1682


- List of User 1: [Ratings of all the movies by User 1]
- List of User 2: [Ratings of all the movies by User 2]

+ Define a function to use it to convert both of the training and test data

#### In the below function some points to explain

- what we are trying to do here is to make a list of lists and every list correspond to a specific user and every element in the list correspondse to the specific movie  

- first we are creating an empty list
- then loop in range of the number of users in our data and in every step we create a list of length of the number of movies and every value will represent a rate for that movie.
- For every user that didn't rate a movie we will put a zero on that element
- and for every rate we put it in the list

In [7]:
def convert_fn(data):
    converted_data = []
    
    for user_id in range(1, nb_users + 1):
        # getting all the movies ids that rated by every user
        movies_for_user_id = data[:, 1][data[:, 0] == user_id] # getting the movies ids that taken be the current user in the for loop
        rating_for_user_id = data[:, 2][data[:, 0] == user_id]
        
        ratings = np.zeros(nb_movies) # initialze all the ratings with zeros then include the rated movies
        ratings[movies_for_user_id - 1] = rating_for_user_id
        converted_data.append(list(ratings))
        
    return converted_data        

- To chk if our function working let's print some data to get some intuations

In [8]:
print(f'That is the rating value:{training_set[0, 2]}, for the movie numeber:{training_set[0, 1]} from the user number:{training_set[0, 0]}')
print(f'That is the rating value:{training_set[1, 2]}, for the movie numeber:{training_set[1, 1]} from the user number:{training_set[1, 0]}')
print(f'That is the rating value:{training_set[2, 2]}, for the movie numeber:{training_set[2, 1]} from the user number:{training_set[2, 0]}')

That is the rating value:3, for the movie numeber:2 from the user number:1
That is the rating value:4, for the movie numeber:3 from the user number:1
That is the rating value:3, for the movie numeber:4 from the user number:1


#### Some intuations from the previous prints:
- the user didn't rate movie \#1 as the first row represent the movie \#2
    - So the first element in the first list should be 0
- the begging of the first list should look like this \[0, 3, 4, 3...\]
- so let's convert them and see the results

In [9]:
new_training = convert_fn(training_set)
new_test = convert_fn(test_set)

Printing the first **list** from the data set

In [10]:
print(new_training[0][:10]) #printing the first ten elemnts in the first list
print(f'\nThe dimensions of the data is {len(new_training)} X {len(new_training[0])}')

[0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0]

The dimensions of the data is 943 X 1682


So as we expected the first four values are true and the dimensions are good
- So we good to go

### Convert the data into torch tesnors

In [11]:
training_d = torch.FloatTensor(new_training)
test_d = torch.FloatTensor(new_test)

### The last step in data preprocessing:
- Is to convert all the rating into **Binary rating** `1` if it's a positive rating and `0` if it's negative and `-1` for movies that didn't get rated.
- Positive movies have rate value from (3 to five).
- negative movies have rate value (1 or 2).

In [12]:
# Training_set
training_d[training_d == 0] = -1 # movies that didn't get rated
training_d[training_d == 1] = 0 
training_d[training_d == 2] = 0 
training_d[training_d >= 3] = 1 

# Test_set
test_d[test_d == 0] = -1 # movies that didn't get rated
test_d[test_d == 1] = 0 
test_d[test_d == 2] = 0 
test_d[test_d >= 3] = 1 


---
---

# 3. Creating Our RBM Architcture

1. First we are gonna creat the RBM class
Inside this class we are goining to create 3 functions
    - \_\_init\_\_ to initialize the weight and biases
    - 

In [13]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.bias_h = torch.randn(1, nh) # bias for the hidden nodes 
        self.bias_v = torch.randn(1, nv) # bias for the visible nodes 
        
    def sample_h(self, x): # x -> The visible neurons in the probablities of p(h) given v
        wx = torch.mm(x, self.W.t())
        activation = wx + self.bias_h.expand_as(wx) # using expaned_as to make the dimensions consistent
        p_hidden_given_visible = torch.sigmoid(activation)
        
        return p_hidden_given_visible, torch.bernoulli(p_hidden_given_visible)
    
    def sample_v(self, y): # x -> The visible neurons in the probablities of p(h) given v
        wy = torch.mm(y, self.W)
        activation = wy + self.bias_v.expand_as(wy) # using expaned_as to make the dimensions consistent
        p_visible_given_hidden = torch.sigmoid(activation)
        
        return p_visible_given_hidden, torch.bernoulli(p_visible_given_hidden)
    
    # train what we are doing here is to update the weights and biases 
    # Using contrastive divergence 
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.bias_v += torch.sum((v0 - vk), 0)
        self.bias_h += torch.sum((ph0 - phk), 0)
                

### Creating our object


In [14]:
nv = len(training_d[0]) 
print(len(training_d[0]) )
nh = 100 # parameter to tune to try getting the best results
batch_zs = 100
rbm = RBM(nv, nh)

1682


In [21]:
nb_epochs = 10
for epoch in range(1, nb_epochs + 1):
    train_loss = 0
    counter = 0.
    
    for id_usr in range(0, nb_users - batch_zs, batch_zs):
        vk = training_d[id_usr:id_usr+batch_zs] # that is our input
        v0 = training_d[id_usr:id_usr+batch_zs] # that is our target the same as the input but not updated
        ph0, _ = rbm.sample_h(v0)
        
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0] # avoid updating the the non rated movies
        
        phk, _ = rbm.sample_h(vk)
        # then call the train function to update the weights and biases
        rbm.train(v0, vk, ph0, phk)
        
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0])) # taking the difference between the target and the input
        counter += 1.
    print(f'Epoch: {epoch}, loss: {train_loss / counter}')
        
        

Epoch: 1, loss: 0.2498481571674347
Epoch: 2, loss: 0.245169535279274
Epoch: 3, loss: 0.24612638354301453
Epoch: 4, loss: 0.24486500024795532
Epoch: 5, loss: 0.2459617406129837
Epoch: 6, loss: 0.24962538480758667
Epoch: 7, loss: 0.2453438639640808
Epoch: 8, loss: 0.24563489854335785
Epoch: 9, loss: 0.2427835464477539
Epoch: 10, loss: 0.24556803703308105


### Testing our model

In [20]:
test_loss = 0
counter = 0.

for id_usr in range(nb_users):
    v = training_d[id_usr:id_usr+1] # that is our input
    vt = test_d[id_usr:id_usr+1] # that is our target the same as the input but not updated
    
    if len(vt[vt >= 0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)

        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0])) # taking the difference between the target and the input
        counter += 1.0
print(f'test loss: {test_loss / counter}')


test loss: 0.2568473815917969


So we get about 75% of success.

**Hope you enjoyed it,  
Thanks for reading.  
Peace ^_^**